In [ ]:
pip install pyarrow pandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.translate.bleu_score import corpus_bleu
!pip install rouge
from rouge import Rouge

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

!pip install transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
!pip install torch torchvision
import torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
batch_size=2000

pqfile=pq.ParquetFile('/content/sample_data/cnn_dailymail-train-00001-of-00003.parquet')

# Read the Parquet file into a Pandas DataFrame
df = pqfile.read().to_pandas()

# Calculate the desired size of the subset (50% of the original dataset)
subset_size = int(len(df) * 0.05)

# Take a subset
train = df[:subset_size]

print(train.head())


                                             article  \
0  By . Harriet Arkell . PUBLISHED: . 07:55 EST, ...   
1  PUBLISHED: . 14:12 EST, 1 April 2013 . | . UPD...   
2  By . Harriet Arkell . PUBLISHED: . 10:53 EST, ...   
3  By . Harriet Arkell . PUBLISHED: . 13:17 EST, ...   
4  By . Harriet Arkell . PUBLISHED: . 12:35 EST, ...   

                                          highlights  \
0  Mark Duper, 54, faces child felony abuse charg...   
1  Fence change for this year's race after accusa...   
2  Six-and-a-half foot snake has been missing fro...   
3  Rhianna Thomas, 24, from Abingdon, denies poss...   
4  Ghassan Hitto, 50, voted new head of Syrian Na...   

                                         id  
0  a03a18be2af9e301a8edc42cfe6ab83c2c284ad1  
1  906a80db7e88d4b7701bebca7d837f8cdfe9ca94  
2  12b801c3da25330cd2b617246607a7ec5e033dbf  
3  904093d925fe423274187bbe2fb112b2442d726f  
4  94fd1ac9f9ba158c7edf6eb78d78d4fedb36d079  


In [ ]:
# Get the total number of rows in the DataFrame
total_rows = train.shape[0]
print(total_rows)


5800


In [ ]:
# Initialize the stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
# Process the data in batches
for start in range(0, len(train), batch_size):
  batch = df[start:start+batch_size]

        # Tokenize, remove stop words, and lemmatize the text
  processed_texts = []
  for text_column in batch.columns:
    tokens = word_tokenize(str(batch[text_column]))
    filtered_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stop_words]
    processed_text = ' '.join(filtered_tokens)
    processed_texts.append(processed_text)



In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Tokenize the processed texts
tokenized_texts = tokenizer.batch_encode_plus(
    processed_texts,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors='pt'
)

# Prepare the input tensors
input_ids = tokenized_texts['input_ids']
attention_mask = tokenized_texts['attention_mask']

# Train the summarization model
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Epoch: 1, Loss: 11.613097190856934
Epoch: 2, Loss: 11.329151153564453


In [ ]:
import pandas as pd

In [ ]:
from transformers.data.data_collator import DataCollatorForSeq2Seq
#Load the test dataset

pqtest=pq.ParquetFile('/content/sample_data/cnn_dailymail-test.parquet')

# Read the Parquet file into a Pandas DataFrame
df2 = pqtest.read().to_pandas()

# Calculate the desired size of the subset (50% of the original dataset)
subset_size = int(len(df2) * 0.3)

# Take a subset of the shuffled dataset
test = df2[:subset_size]

print(test.head())

In [ ]:
#Loading the all highlights test labels for each aticle
#This test labels are the summaries of the respective article
test_labels =test['highlights'].tolist()

In [ ]:
# Get the total number of rows in the DataFrame
total_rows = test.shape[0]
print(total_rows)


In [ ]:
# Process the data in batches
for start in range(0, len(test), batch_size):
  batch = df[start:start+batch_size]

        # Tokenize, remove stop words, and lemmatize the text
  processed_texts2 = []
  for text_column in batch.columns:
    tokens = word_tokenize(str(batch[text_column]))
    filtered_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stop_words]
    processed_text = ' '.join(filtered_tokens)
    processed_texts2.append(processed_text)

In [ ]:
# Prepare the input tensors for the test dataset
test_tokenized_texts = tokenizer.batch_encode_plus(
    processed_texts2,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors='pt'
)

test_input_ids = test_tokenized_texts['input_ids']
test_attention_mask = test_tokenized_texts['attention_mask']


In [ ]:
# Evaluation on test dataset
model.eval()
with torch.no_grad():
    test_outputs = model.generate(input_ids=test_input_ids, attention_mask=test_attention_mask)
    test_predicted_summaries = tokenizer.batch_decode(test_outputs, skip_special_tokens=True)

# Check the lengths of test_predicted_summaries and test_reference_summaries
print(f"Number of predicted summaries: {len(test_predicted_summaries)}")
print(f"Number of reference summaries: {len(test_reference_summaries)}")

# Calculate BLEU score for test dataset
test_reference_summaries = test_labels  # Convert tensor to list of summaries
bleu_score = corpus_bleu([[ref_summary] for ref_summary in test_reference_summaries], [test_predicted_summaries])

# Calculate ROUGE scores for test dataset
rouge = Rouge()
rouge_scores = rouge.get_scores(test_predicted_summaries, test_reference_summaries, avg=True)

# Print the evaluation results
print(f"BLEU Score: {bleu_score}")
print(f"ROUGE Scores: {rouge_scores}")